In [5]:
import os
import numpy as np
from scipy.signal import butter, iirnotch, lfilter
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import neurokit2 as nk
import pandas as pd
from ecgdetectors import Detectors
import pyhrv.frequency_domain as fd
import pyhrv.time_domain as td
import pyhrv
import tensorflow as tf

In [15]:
# Function that reads a csv file and outputs a data frame
def read_from_csv(file):
    data_frame = pd.read_csv(file, sep=",")
    return data_frame

In [16]:
df_features = read_from_csv('out_features.csv')
df_features.head()
y = []
for i in range(15):
    y.extend([1,2,1,1])

print(len(y))
print(y)


60
[1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1]


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_features, y, test_size=0.3, random_state=4)

# Scale the data
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Apply PCA
pca = PCA(n_components=14)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

# Convert labels to categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Define the feedforward neural network
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(14,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
from tensorflow.keras.optimizers import Adam
# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train_pca, y_train, validation_data=(X_test_pca, y_test), epochs=75, batch_size=100, verbose=2)
val_accuracies = []
for i in range(10):
    # ... your code ...
    history = model.fit(X_train_pca, y_train, validation_data=(X_test_pca, y_test), epochs=10, batch_size=32, verbose=2)
    val_accuracy = history.history['val_accuracy'][-1]
    val_accuracies.append(val_accuracy)

avg_val_accuracy = sum(val_accuracies) / len(val_accuracies)
print("Average Validation Accuracy:", avg_val_accuracy)


   Unnamed: 0  feat_nn20  feat_nn50  feat_rmssd   feat_nnhr   feat_sdsd  \
22        S73      275.0      163.0    0.076788   58.710806   78.965915   
33       S102      273.0       64.0    0.063904  108.861348   35.787715   
15        S54      369.0       84.0    0.034627   64.224175   32.497720   
29        S92      481.0      174.0    0.084124   80.729926   63.333814   
18        S63      250.0       75.0    0.046496   66.343922   42.363071   
41       S132      234.0       43.0    0.037942  109.772265   20.998153   
27        S84      683.0      372.0    0.089754   70.722119   77.596404   
31        S94      484.0      135.0    0.055477   72.767872   46.114963   
57       S172      309.0      111.0    0.073406  115.732132   38.850940   
13        S52      189.0       12.0    0.095763   90.134331   64.318413   
10        S43      288.0      171.0    0.082060   62.428316   79.565409   
19        S64      600.0      247.0    0.064856   67.494824   58.261392   
34       S103      267.0 

ValueError: could not convert string to float: 'S73'